# Building Neural Networks
## Importing libraries

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# What device for training?
Use GPU of MPS if available. Device configured as such.

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Defining the nn class
Initialise the neural network layers in `__init__`. Every NN subclass implements the operations on input data in the `forward` method

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Create an instance of the class move it to the device and check the structure

In [5]:

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


As a test call the model with so

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


# Layers
Break down the layers in the FashionMNIST model. Showing this taking a sample of 3 images of size $28 \times 28$

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# `nn.Flatten`
Initialising the `nn.flatten` layer turns each 2d 28x28 imsgine into a contiguous array of 784 pixel values
(minibatch dimension(at dim = 0) is maintained)

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# `nn.Linear`
Applies a linear transformation on the input usin gits stored weights and biases

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# `nn.ReLU`
Non-linear activations are what creates the complex mappings between inputs and outputs. Applied after linear transformations ->introduces nonlinearity
NB : we use `nn.ReLU` between linear layers.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3023,  0.0073,  0.0824,  0.0167,  0.1658, -0.6901,  0.3428,  0.1891,
         -0.4992, -0.3327, -0.0547,  0.3484,  0.0457, -0.5326,  0.2573, -0.0802,
         -0.0663, -0.1082, -0.0534,  0.1303],
        [-0.0995, -0.0548,  0.7120, -0.1790, -0.1979, -0.3309,  0.6175,  0.1577,
         -0.9651, -0.4283,  0.0623,  0.2574, -0.0728, -0.6717,  0.1305, -0.2182,
         -0.2958, -0.2314,  0.1740,  0.1030],
        [-0.4701, -0.0150,  0.7958,  0.0579,  0.0675, -0.5418,  0.3622,  0.1876,
         -0.2196, -0.3656, -0.2660,  0.0188, -0.2413, -0.4393,  0.2849,  0.0102,
         -0.1003, -0.2320,  0.3071,  0.1246]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0073, 0.0824, 0.0167, 0.1658, 0.0000, 0.3428, 0.1891, 0.0000,
         0.0000, 0.0000, 0.3484, 0.0457, 0.0000, 0.2573, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1303],
        [0.0000, 0.0000, 0.7120, 0.0000, 0.0000, 0.0000, 0.6175, 0.1577, 0.0000,
         0.0000, 0.0623, 0.2574, 0.0000, 0.0000, 0.13

# `nn.Sequential`
An ordered container of modules. Data passed throuugh all the modules in the order defined. Sequential containers are used to put together a quick network like `seq_modules`.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# `nn.Softmax`
The last layer returns logits $\in [-\infty,\infty]$. These are passed to the `nn.Softmax` module. Logits scaled to values $[0,1]$ representing the models predicted probabilities for each class. `dim` parameter indicated the dimension along which the values sum to 1

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1238, 0.0799, 0.0964, 0.1047, 0.1076, 0.0940, 0.1022, 0.1036, 0.0860,
         0.1020],
        [0.1279, 0.0872, 0.0841, 0.1115, 0.1049, 0.0849, 0.1046, 0.0985, 0.0851,
         0.1113],
        [0.1313, 0.0857, 0.0859, 0.0991, 0.1122, 0.0946, 0.1030, 0.1060, 0.0820,
         0.1003]], grad_fn=<SoftmaxBackward0>)


# Model Parameters
Layers inside nn are parameterized (have associated weights and biases). Subclassing `nn.Module` automatically tracks all fields defined inside the object and makes all parameters accessible using the models `parameters()` or `named_parameters()` methods/

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0163, -0.0223,  0.0036,  ...,  0.0089,  0.0022, -0.0111],
        [ 0.0026, -0.0120,  0.0008,  ..., -0.0019,  0.0354, -0.0164]],
       grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0215, 0.0007], grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0170, -0.0130,  0.0379,  ...,  0.0332,  0.0358, -0.0108],
        [-0.0229,  0.0243,  0.0127,  ...,  0.0309, -0.0391, -0.0178]],
       grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.2.bias | Size: torch